# Temporal-Difference (TD) Learning

If one had to identify one idea as central and novel to reinforcement learning, it would undoubtedly be **temporal-difference (TD) learning**

TD learning is a combination of Monte Carlo ideas and Dynamic Programming ideas.

> Like Monte Carlo methods, TD methods can learn directly from raw experience without a model of the environment’s dynamics.

> Like DP, TD methods update estimates based in part on other learned estimates, without waiting for a ﬁnal outcome.

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from collections import defaultdict
from IPython.display import clear_output

## Q-learning

The state-action value update is given by:

$$
Q({S_t},{A_t}) \leftarrow Q({S_t},{A_t}) + \alpha \left[{R_{t+1}} + \gamma \max\limits_a Q({S_{t+1}},{a}) - Q({S_t},{A_t})\right]
$$

In [ ]:
class QLearningAgent:
    def __init__(self, alpha, epsilon, discount, get_legal_actions):
        """
        Q-Learning Agent
        Instance variables you have access to
          - self.epsilon (exploration prob)
          - self.alpha (learning rate)
          - self.discount (discount rate aka gamma)

        Functions you should use
          - self.get_legal_actions(state) {state, hashable -> list of actions, each is hashable}
            which returns legal actions for a state
          - self.get_qvalue(state, action), which returns Q(state,action)
          - self.set_qvalue(state, action, value), which sets Q(state,action) := value
        """

        self.get_legal_actions = get_legal_actions
        self._qvalues = defaultdict(lambda: defaultdict(lambda: 0))
        self.alpha = alpha
        self.epsilon = epsilon
        self.discount = discount

    def get_qvalue(self, state, action):
        """ Returns Q(state,action) """
        return self._qvalues[state][action]

    def set_qvalue(self, state, action, value):
        """ Sets the Qvalue for [state, action] to the given value """
        self._qvalues[state][action] = value

    def get_value(self, state):
        """
        Compute your agent's estimate of V(s):
            V(s) = max_over_action Q(state, action).
        """
        possible_actions = self.get_legal_actions(state)  # please take into account that q-values can be negative

        # If there are no legal actions, return 0.0
        if len(possible_actions) == 0:
            return 0.0
        
        value = max(self.get_qvalue(state, possible_action) for possible_action in possible_actions)
        
        return value

    def update(self, state, action, reward, next_state):
        """
        You should do your Q-Value update here:
            Q(s,a) := (1 - alpha) * Q(s,a) + alpha * (r + gamma * V(s'))
        """

        # agent parameters
        gamma = self.discount
        lr = self.alpha  # learning rate
        
        qvalue = (1 - lr) * self.get_qvalue(state, action) + lr * (reward + gamma * self.get_value(next_state))
        
        self.set_qvalue(state, action, qvalue)

    def get_best_action(self, state):
        """
        Compute the best action to take in a state (using current q-values).
        """
        possible_actions = self.get_legal_actions(state)

        # If there are no legal actions, return None
        if len(possible_actions) == 0:
            return None
        
        q_dict = {possible_action: self.get_qvalue(state, possible_action) for possible_action in possible_actions}
    
        max_q = max(q_dict.values())
        best_actions = [action for action, q in q_dict.items() if q == max_q]
        best_action = random.choice(best_actions)

        return best_action

    def get_action(self, state):
        """
        Compute the action to take in the current state, including exploration.  
        With probability self.epsilon, we should take a random action.
            otherwise - the best policy action (self.get_best_action).
        """

        # Pick Action
        possible_actions = self.get_legal_actions(state)
        action = None

        # If there are no legal actions, return None
        if len(possible_actions) == 0:
            return None

        # agent parameters
        epsilon = self.epsilon
        
        if random.random() < self.epsilon:
            chosen_action = random.choice(possible_actions)
        else:
            chosen_action = self.get_best_action(state)

        return chosen_action

## SARSA
The state-action value update is given by:

$$
Q({S_t},{A_t}) \leftarrow Q({S_t},{A_t}) + \alpha \left[{R_{t+1}} + \gamma Q({S_{t+1}},{A_{t+1}}) - Q({S_t},{A_t})\right]
$$

In [ ]:
class SarsaAgent(QLearningAgent):
    """ SARSA Agent """
    
    def sarsa_update(self, state, action, reward, next_state,next_action):
        """
        You should do your Q-Value update here:
           Q(s,a) := (1 - alpha) * Q(s,a) + alpha * (r + gamma * V(s'))
           Q(s,a) := Q(s,a) + alpha(R+ gamma * Q (s',a') - Q(s,a))
        """
        # agent parameters
        gamma = self.discount
        learning_rate = self.alpha
        
        qvalue = (1 - learning_rate) * self.get_qvalue(state, action) + learning_rate *\
        (reward + gamma * self.get_qvalue(next_state, next_action))
        
        self.set_qvalue(state, action, qvalue)

## Expected Value SARSA

**Expected Value SARSA** is an online algorithm - instead of taking the maximum action value for a state (in Q learning) it takes  expectations of the all the actions in a state.

The state-action value update is given by:
$$
Q({S_t},{A_t}) \leftarrow Q({S_t},{A_t}) + \alpha \left[{R_{t+1}} + \gamma \sum_{a}\pi(a|S_{t+1})Q({S_{t+1}},a) - Q({S_t},{A_t})\right]
$$

In [ ]:
class EVSarsaAgent(QLearningAgent):
    """ Expected Value SARSA Agent """
    
    def get_value(self, state):
        """ 
        Returns Vpi for current state under epsilon-greedy policy:
          V_{pi}(s) = sum _{over a_i} {pi(a_i | s) * Q(s, a_i)}
        """
        epsilon = self.epsilon
        possible_actions = self.get_legal_actions(state)

        # if there are no legal actions, return 0.0
        if len(possible_actions) == 0:
            return 0.0
        
        len_actions = len(possible_actions)
        
        state_value = 0
        for possible_action in possible_actions:
            if possible_action == self.get_best_action(state):
                state_value += (epsilon / len_actions + (1 - epsilon)) * self.get_qvalue(state, possible_action)
            else:
                state_value += (epsilon / len_actions * self.get_qvalue(state, possible_action))
        
        return state_value

## Expected Value Softmax SARSA

$$ \pi(a_i|s) = softmax({Q(s,a_i) \over \tau}) = {e ^ {Q(s,a_i)/ \tau}  \over {\sum_{a_j}  e ^{Q(s,a_j) / \tau }}} $$

* Sample <s,a,r,s'> from environment
* Compute $\hat{Q}(s,a) = r(s,a) + \gamma\mathbb{E}_{a_i \sim\pi(a|s')}Q(s',a_i)$ with a probability of {a_i \sim\pi(a|s')} being **softmax policy**
* Update $Q(s,a) \leftarrow (1-\alpha)*Q(s,a) + \alpha*\hat{Q}(s,a)$

In [ ]:
class EVSoftmaxSarsaAgent(QLearningAgent):
    """ Expected Value Softmax SARSA Agent """

    def get_value(self, state):
        epsilon = self.epsilon
        possible_actions = self.get_legal_actions(state)

        # if there are no legal actions, return 0.0
        if len(possible_actions) == 0:
            return 0.0
        
        qvalues = np.array([self.get_qvalue(state, possible_action) for possible_action in possible_actions])

        exp_sum = sum(np.exp(qvalues))
        probabilities = [np.exp(q) / exp_sum for q in qvalues]
        
        state_value = sum(probabilities[i] * qvalues[i] for i in range(len(possible_actions)))
        
        return state_value

## Play and Train Processes

In [ ]:
def play_and_train(env, agent, t_max=10**4):
    total_reward = 0.0
    state = env.reset()

    for t in range(t_max):
        # get agent to pick action given state s.
        action = agent.get_action(state)

        next_state, reward, done, _ = env.step(action)

        # train (update) agent for state s
        agent.update(state, action, reward, next_state)
        
        state = next_state
        total_reward += reward
        if done:
            break
    
    return total_reward

In [ ]:
def play_and_train_sarsa(env, agent, t_max=10**4):
    total_reward = 0.0
    state = env.reset()
    action = agent.get_action(state)  # this is the difference from Q learning
    
    for t in range(t_max):
        # get agent to pick action given state
        next_state, reward, done, _ = env.step(action)
        next_action = agent.get_action(next_state)  # this is another difference from Q learning
        
        agent.sarsa_update(state, action, reward, next_state, next_action)
        
        state, action = next_state, next_action
        total_reward += reward
        if done:
            break

    return total_reward

## Comparison of Temporal-Difference Learning Algorithms

In [ ]:
import gym, gym.envs.toy_text

env = gym.envs.toy_text.CliffWalkingEnv()   # print(env.__doc__)

n_actions = env.action_space.n

In [ ]:
# configuration
considered_epsilon = 0.2

agent_ql = QLearningAgent(alpha=0.25, epsilon=considered_epsilon, discount=0.99,
                          get_legal_actions=lambda s: range(n_actions))

agent_sarsa = SarsaAgent(alpha=0.25, epsilon=considered_epsilon, discount=0.99,
                           get_legal_actions=lambda s: range(n_actions))

agent_ev_sarsa = EVSarsaAgent(alpha=0.25, epsilon=considered_epsilon, discount=0.99,
                           get_legal_actions=lambda s: range(n_actions))

agent_softmax = EVSoftmaxSarsaAgent(alpha=0.25, epsilon=considered_epsilon, discount=0.99,
                          get_legal_actions=lambda s: range(n_actions))

In [ ]:
def moving_average(x, span=100):
    return pd.DataFrame({'x': np.asarray(x)}).ewm(span=span).mean().values

rewards_ql, rewards_sarsa, rewards_ev_sarsa, rewards_softmax = [], [], [], []

for i in range(10_000):
    rewards_ql.append(play_and_train(env, agent_ql))
    rewards_sarsa.append(play_and_train_sarsa(env, agent_sarsa))
    rewards_ev_sarsa.append(play_and_train(env, agent_ev_sarsa))
    rewards_softmax.append(play_and_train(env, agent_softmax))
    
    if i % 100 == 0:
        clear_output(True)
        print('QLEARNING mean reward =', np.mean(rewards_ql[-100:]))
        print('SARSA mean reward =', np.mean(rewards_sarsa[-100:]))
        print('EVSARSA mean reward =', np.mean(rewards_ev_sarsa[-100:]))
        print('SOFTMAX mean reward =', np.mean(rewards_softmax[-100:]))
        
        plt.title("epsilon = %s" % agent_ql.epsilon)
        plt.plot(moving_average(rewards_ql), label='Q-learning')
        plt.plot(moving_average(rewards_sarsa), label='SARSA')
        plt.plot(moving_average(rewards_ev_sarsa), label='Expected Value SARSA')
        plt.plot(moving_average(rewards_softmax), label='Expected Value Softmax SARSA')
        plt.grid()
        plt.legend()
        plt.ylim(-500, 0)
        plt.show()

## Reference

1. Practical RL: Higher School of Economics (HSE), Yandex. Link: https://github.com/yandexdataschool/Practical_RL
2. Richard S. Sutton and Andrew G. Barto, "Reinforcement Learning: An Introduction", The MIT Press, Cambridge, Massachusetts, London, England